### **Introduction**

Welcome to your first official coding Assignment of 4756. In this short notebook, we are going to train agents to play the LunarLander game from OpenAI Gym, using data provided by an expert agent. Specifically, you will gain exposure to implementing behavioral cloning (BC) and dataset aggregation (DAgger) methods.

**Evaluation:**
Your code will be tested for correctness, and for certain assignments, speed. Please remember that all assignments should be completed individually.

**Academic Integrity:** We will be checking your code against other submissions in the class for logical redundancy. If you copy someone else’s code and submit it with minor changes, we will know. These cheat detectors are quite hard to fool, so please don’t try. We trust you all to submit your own work only; please don’t let us down. If you do, we will pursue the strongest consequences available to us.

**Getting Help:** The [#resources](https://www.cs.cornell.edu/courses/cs4756/2023sp/#resources) section on the course website is your friend (especially for this first assignment)! If you ever feel stuck in these projects, please feel free to avail yourself to office hours and Edstem! If you are unable to make any of the office hours listed, please let TAs know and we will be happy to assist. Since this is the first iteration of this course, please do not hesitate to reach out to TAs if you find any errors in the assignments. 

### Preliminaries

In this assignment we will be using modules and agents from OpenAI Gym. Please run the following cells to make sure this notebook is properly configured. You should only need to run the following cell once, so feel free to comment it out after it has been installed the first time. 

If the `pip install` leaves you with any messages at the bottom telling you to install more packages, please feel free to add those lines in a separate cell. 

In [1]:
!pip3 install gym[box2d]
!pip3 install -q stable-baselines3[extra]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 KB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Failed to build box2d-py
  Running setup.py install for box2d-py ... done
  DEPRECATION: box2d-py was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. Discussion can be found at https:

In [2]:
# Part 1 imports
from typing import Type, List
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

# Part 2 imports
from torch import optim

# Part 3 imports
import gym
from stable_baselines3.ppo import PPO
import torch.nn as nn
import argparse

Please run the following cells to get the expert data onto your noteboook. Then run `!ls` and verify that the file "lunarlander_expert.zip" exits



In [3]:
!wget -nc https://github.com/portal-cornell/cs4756_robot_learning/blob/main/assignments/HW1/LunarLander-v2/lunarlander_expert.zip?raw=true

--2023-02-23 03:44:59--  https://github.com/portal-cornell/cs4756_robot_learning/blob/main/assignments/HW1/LunarLander-v2/lunarlander_expert.zip?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/portal-cornell/cs4756_robot_learning/raw/main/assignments/HW1/LunarLander-v2/lunarlander_expert.zip [following]
--2023-02-23 03:44:59--  https://github.com/portal-cornell/cs4756_robot_learning/raw/main/assignments/HW1/LunarLander-v2/lunarlander_expert.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/portal-cornell/cs4756_robot_learning/main/assignments/HW1/LunarLander-v2/lunarlander_expert.zip [following]
--2023-02-23 03:44:59--  https://raw.githubusercontent.com/portal-cornell/cs4756_robot_learning/main/assignments/HW1/LunarLander-v2/lunarlande

In [4]:
!mv lunarlander_expert.zip?raw=true lunarlander_expert.zip
!ls | grep "lunar"

lunarlander_expert.zip


To verify that everything has been downloaded correctly, please run the following cell and check for errors. If none appear, you're good to go! (You may ignore the warnings.warn() messages)

In [5]:
env = gym.make("LunarLander-v2")
if PPO.load("./lunarlander_expert"):
    print("Success!")

Success!


/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)
  warnings.warn(


**Please make sure there are no errors in the above import statements before continuing to the rest of the assignment**

### Part 1: Simple utilities for later use

In the following section, we will define most of the helper methods that will become useful to you in training and evaluating you imitation agents. There are three sets of utility functions: "NEURAL NET UTILS", "ENV UTILS", and "EVAL UTILS." Please take the time to understand what each function is doing, and also implement the `argmax_policy()` function in the second cell below. 

**Note: you may ignore all functions that deal with truncate until you get to the extra credit.**

In [6]:
# ====== NEURAL NET UTILS ======

def create_mlp(input_dim: int, output_dim: int, architecture: List[int], squash=False, activation: Type[nn.Module]=nn.ReLU) -> List[nn.Module]:
    '''Creates a list of modules that define an MLP.'''
    if len(architecture) > 0:
        layers = [nn.Linear(input_dim, architecture[0]), activation()]
    else:
        layers = []
        
    for i in range(len(architecture) - 1):
        layers.append(nn.Linear(architecture[i], architecture[i+1]))
        layers.append(activation())
    
    if output_dim > 0:
        last_dim = architecture[-1] if len(architecture) > 0 else input_dim
        layers.append(nn.Linear(last_dim, output_dim))
        
    if squash:
        # squashes output down to (-1, 1)
        layers.append(nn.Tanh())
    
    return layers

def create_net(input_dim: int, output_dim: int, squash=False):
    layers = create_mlp(input_dim, output_dim, architecture=[64, 64], squash=squash)
    net = nn.Sequential(*layers)
    return net

def argmax_policy(net):
    # TODO: Return a FUNCTION that takes in a state, passes it through the network, and outputs index of the action with the highest probability.
    # Inputs:
    # - net: (type nn.Module). A neural network module, going from state dimension to number of actions.
    # Wanted output:
    # - argmax_fn: A function which takes in a state, and outputs argmax of the action vector.
    # return torch.argmax(net)

    def argmax_fn(state):
        state = torch.from_numpy(state).float() # convert state to tensor and float type so it can be passed through net
        values = net(state) # pass through net to get values for each action in state
        max_value_index = torch.argmax(values) # get the maximum value from the values for each action in state
        return max_value_index
        
    return argmax_fn

def expert_policy(expert, s):
    '''Returns a one-hot encoded action of what the expert predicts at state s.'''
    action = expert.predict(s)[0]
    one_hot_action = np.eye(4)[action]
    return one_hot_action

In [7]:
# ====== ENV UTILS ======

def rollout(net, env, truncate=True):
    '''Rolls out a trajectory in the environment, with optional state masking.'''
    states = []
    actions = []
    
    ob = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        states.append(ob.reshape(-1))
        ob_tensor = torch.from_numpy(np.array(ob))
        if truncate:
            action = net(ob_tensor[:-3].float())
        else:
            action = net(ob_tensor.float())
            
        # detach action and convert to np array
        if isinstance(action, torch.FloatTensor) or isinstance(action, torch.Tensor):
            action = action.detach().numpy()
        actions.append(action.reshape(-1))
        
        # step env
        ob, r, done, _ = env.step(np.argmax(action))
        total_reward += r
        
    states = np.array(states, dtype='float')
    actions = np.array(actions, dtype='float')
    return states, actions

def expert_rollout(expert, env, truncate=False):
    '''Rolls out an expert trajectory in the environment, with optional state masking.'''
    expert_net = lambda s: expert_policy(expert, s)
    return rollout(expert_net, env, truncate=truncate)

In [8]:
# ====== EVAL UTILS ======

def eval_policy(policy, env, truncate=True):
    '''Evaluates policy with one trajectory in environment. Returns accumulated reward.'''
    done = False
    ob = env.reset()
    total_reward = 0
    while not done:
        if truncate:
            action = policy(ob[:-2])
        else:
            action = policy(ob)
        
        # detach action and convert to np array
        if isinstance(action, torch.FloatTensor) or isinstance(action, torch.Tensor):
            action = action.detach().numpy()
        
        # step env and observe reward
        ob, r, done, _ = env.step(action)
        total_reward += r
    
    return total_reward

### Part 2: Behavioral Cloning & DAgger

It is now time to build up our agents! Please read the directions carefully, and avail yourself to the myriad of resources in this class if you feel stuck!

**Behavioral cloning:** Behavioral cloning is the simplest imitation learning algorithm, where we perform supervised learning on the given (offline) expert dataset. We either do this via log likelihood maximization (cross entropy minimization) in the discrete action case, or mean-squared error minimization (can also do MLE) in the continuous control setting.

Please implement the following `learn()` function for BC.

In [9]:
class BC:
    def __init__(self, net, loss_fn):
        self.net = net
        self.loss_fn = loss_fn

        self.states_history = [] #past history for states
        self.actions_history = [] #past history for actions
        
        self.opt = optim.Adam(self.net.parameters(), lr=3e-4)
        
    def learn(self, env, states, actions, val_states, val_actions, val_steps=10000, n_steps=1e4, truncate=True):
        # TODO: Implement this method. Return the final greedy policy (argmax_policy).

        states = torch.from_numpy(states).float() # convert states to tensor and float type
        actions = torch.from_numpy(actions).float() # convert actions to tensor and float type

        val_states = torch.from_numpy(val_states).float() # convert validation states to tensor and float type
        val_actions = torch.from_numpy(val_actions).float() # convert validation actions to tensor and float type


        loss_list = [] # list of loss values
        reward_list = [] # list of reward values    
        val_loss_list = [] # list of validation loss values
        for i in range(1, n_steps+1): # loop through n_steps
            self.opt.zero_grad() # zero out the gradients
            outputs = self.net(states) # pass through net to get values for each action in state
            loss = self.loss_fn(outputs, actions) # calculate loss
            loss.backward() # backpropagate
            self.opt.step() # update weights
            loss_list.append(loss.item()) # append loss to loss list
            # self.states_history.append(state) # append to the current states
            # self.actions_history.append(actions[j]) # append to the current actions

            if i % val_steps == 0: # if i is divisible by val_steps
                with torch.no_grad(): # turn off gradient tracking
                    val_outputs = self.net(val_states) # pass through net to get values for each action in state
                    val_loss = self.loss_fn(val_outputs, val_actions) # calculate loss
                val_loss_list.append(val_loss.item()) # append loss to loss list
            
            if i % 100 == 0: # if i is divisible by 100
                reward_list.append(np.mean(eval_policy(argmax_policy(self.net), env, truncate)))
                
        return argmax_policy(self.net), loss_list, reward_list, val_loss_list  



**Dataset aggregation (DAgger):** DAgger is a fundamentally interactive algorithm, where we are able to query the expert any time we want to get information about how to proceed. This allows for significantly more freedom for the learner, as it can ask the expert anywhere and not be limited by the dataset that it is given to learn from.

Like BC, please implement the following `learn()` function for DAgger.

In [10]:
class DAgger:
    def __init__(self, net, loss_fn, expert):
        self.net = net
        self.loss_fn = loss_fn
        self.expert = expert

        self.states_history = [] #past history for states
        self.actions_history = [] #past history for actions
        
        self.opt = optim.Adam(self.net.parameters(), lr=3e-4)

    #additional loop to append data;  1. rollout using policy 2. Each state, run expert policy 3. append to tuple 
    def learn(self, env, n_steps=1e4, truncate=True):
        # TODO: Implement this method. Return the final greedy policy (argmax_policy).
        # Make sure you are making the learning process fundamentally expert-interactive.
        self.net.train() # set the network to training mode
        reward_list = [] # list to store the rewards
        loss_list = [] # list to store the loss

        for i in range(1, n_steps+1): # loop over the number of steps to train
            self.opt.zero_grad() # zero the gradients
            expert_states = rollout(self.net, env, truncate)[0] # rollout the policy and get the states

            if truncate: # if truncate is true, remove the last three dimensions, which correspond to angular velocity
                expert_states = expert_states[:, :-3] # remove the last three dimensions, which correspond to angular velocity

            states = torch.from_numpy(expert_states).float() # convert the states to a tensor
            expert_actions = self.net(states) # get the actions from the network
            actions = np.empty((len(expert_states), expert_actions.size(1)), dtype=np.float32) # create an empty array to store the actions
            
            for j, state in enumerate(expert_states): # loop over the states
                actions[j] = expert_policy(self.expert, state) # get the expert action for the state
                # self.states_history.append(state) # append to the current states
                # self.actions_history.append(actions[j]) # append to the current actions

            loss = self.loss_fn(expert_actions, torch.from_numpy(actions)) # compute the loss
            loss.backward() # backpropagate the loss
            self.opt.step() # update the network parameters
            loss_list.append(loss.item()) # append the loss to the loss list

            if i % 100 == 0: # every 100 steps, evaluate the policy and append the reward to the reward list
                reward_list.append(np.mean(eval_policy(argmax_policy(self.net), env, truncate))) 
        return argmax_policy(self.net), reward_list, loss_list



### Part 3: Training loop

Now with the hard part out of the way, it's time to see the performance of your networks! For imitation learning to work, all you need is access to some expert trajectories. The good news is, we've got you covered! 🙂

Please implement the training loop under train() according to the instructions in the code.

In [11]:
def make_env():
    return gym.make("LunarLander-v2")

def get_expert():
    return PPO.load("./lunarlander_expert.zip")

def get_expert_performance(env, expert):
  Js = []
  for _ in range(100):
      obs = env.reset()
      J = 0
      done = False
      hs = []
      while not done:
          action, _ = expert.predict(obs)
          obs, reward, done, info = env.step(action)
          hs.append(obs[1])
          J += reward
      Js.append(J)
  ll_expert_performance = np.mean(Js)
  return ll_expert_performance

In [12]:
def train(train_bc=True, truncate=False, n_steps=10000):
    env = make_env()
    expert = get_expert()
    
    performance = get_expert_performance(env, expert)
    print('=' * 20)
    print(f'Expert performance: {performance}')
    print('=' * 20)
    
    # net + loss fn
    if truncate:
        net = create_net(input_dim=5, output_dim=4)
    else:
        net = create_net(input_dim=8, output_dim=4)
    
    loss_fn = nn.CrossEntropyLoss()
    
    if train_bc:
        # TODO: train BC
        # Things that need to be done:
        # - Roll out the expert for X number of trajectories (a standard amount is 10).
        # - Create our BC learner, and train BC on the collected trajectories.
        # - It's up to you how you want to structure your data!
        # - Evaluate the argmax_policy by printing the total rewards.

        # Create our data so that X number of expert trajectories are rolled out to accumulate data
        n_trajectories = 10
        expert_states, expert_actions = [], []
        for i in range(n_trajectories):
            states_traj, actions_traj = expert_rollout(expert, env)
            expert_states.append(states_traj)
            expert_actions.append(actions_traj)
        # Concatenate the trajectory data into single arrays
        expert_states = np.concatenate(expert_states, axis=0)
        expert_actions = np.concatenate(expert_actions, axis=0)

        #create our validation data
        val_states, val_actions = [], []
        for i in range(n_trajectories):
            states_traj, actions_traj = expert_rollout(expert, env)
            val_states.append(states_traj)
            val_actions.append(actions_traj)

        # Concatenate the trajectory data into single arrays
        val_states = np.concatenate(val_states, axis=0)
        val_actions = np.concatenate(val_actions, axis=0)

        # Create our BC learner, and train BC on the collected trajectories.
        bc_learner = BC(net, loss_fn)
        trained_policy, loss_list, reward_list, val_loss_list = bc_learner.learn(env, expert_states, expert_actions, val_states, val_actions, n_steps=n_steps, truncate=truncate)

        # Evaluate the argmax_policy by printing the total rewards.
        reward = 0
        for i in range(10):
            reward += eval_policy(trained_policy, env, truncate=truncate)
        print("BC Performance: %.12f" % (reward / 10))

        # # Plot the reward over time
        # plt.plot(reward_list)
        # plt.title("Reward over time")
        # plt.xlabel("Training steps")
        # plt.ylabel("Reward")
        # plt.show()

        # # Plot the loss over time
        # plt.plot(loss_list)
        # plt.title("Validation Loss over time")
        # plt.xlabel("Training steps")
        # plt.ylabel("Loss")
        # plt.show()

        # # Plot the validation loss
        # plt.plot(val_loss_list)
        # plt.title("Validation Loss")
        # plt.xlabel("Training steps")
        # plt.ylabel("Loss")
        # plt.show()

    else:
        # TODO: train DAgger
        # Things that need to be done.
        # - Create our DAgger learner.
        # - Set up the training loop. Make sure it is fundamentally interactive!
        # - It's up to you how you want to structure your data!
        # - Evaluate the argmax_policy by printing the total rewards.
        
        # Create our DAgger learner.
        dagger_learner = DAgger(net, loss_fn, expert)

        # Set up the training loop. Make sure it is fundamentally interactive!
        trained_policy, loss_list, reward_list = dagger_learner.learn(env, n_steps, truncate=truncate)

        # Evaluate the argmax_policy by printing the total rewards.
        reward = 0
        for i in range(20):
            reward += eval_policy(trained_policy, env, truncate=truncate)
        print("DAgger Performance: %.12f" % (reward / 20))

        # # Plot the reward over time
        # plt.plot(dagger_learner.reward_list)
        # plt.title("Reward over time")
        # plt.xlabel("Training steps")
        # plt.ylabel("Reward")
        # plt.show()

        # # Plot the loss over time
        # plt.plot(dagger_learner.loss_list)
        # plt.title("Loss over time")
        # plt.xlabel("Training steps")
        # plt.ylabel("Loss")
        # plt.show()
 

In [14]:
train_bc = True
truncate = False
n_steps = 1000

train(train_bc, truncate, n_steps)

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)
  warnings.warn(


Expert performance: 286.82501120676386
BC Performance: 268.228792372148


### Extra Credit:

As a reminder, all extra credit problems are optional for students taking the 4xxx version of this course, but compulsory for students taking the 5xxx version.

Using the `args.truncate` option, create a “partially observable” lunar lander environment where the angular velocity is masked out and not available to the learner (it’s still available to the expert!) You may find yourself needing to add/modify some of the existing code you have, so we recommend saving a version beforehand so that the performance of the fully-observable version is not affected by these changes.


### Writeup

In a separate PDF, please include answers to the questions at the bottom of the Assignment [doc](https://docs.google.com/document/d/1YA8hpE7R8M0prgMtSMKHLdUzWZq3VSoW8mvMOw1EB90/edit#). Double check your work to make sure all cells are running properly. There are two additional answers for you to respond to in the writeup if you did the extra credit. 

**Some questions will ask for an accompanying graph to support your response. Don't forget to include these!**